In [2]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from google.colab import files
from google.colab import drive
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [3]:
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier

In [4]:
d = {'alg': ['LSTM', 'CNN','BERT','albert'],
     'imdb': [.69, .73, .87, .53],#1
     'covid': [.69, .76, .82, .61],#2
     'email': [.55, .48, .52, .54],#3
     'review': [.78, .79, .81, .78],#4
     'twitter': [.65, .64, .74, .54],#5
     'news': [.72, .71, .78, .79],#6
     'reddit': [.63, .43, .60, .50],#7
     'finance': [.62, .58, .59, .47],#8
     'paper': [.70, .69, .70, .62]
     }
     
df = pd.DataFrame(data=d)
df

,alg,imdb,covid,email,review,twitter,news,reddit,finance,paper
0,LSTM,0.69,0.69,0.55,0.78,0.65,0.72,0.63,0.62,0.70
1,CNN,0.73,0.76,0.48,0.79,0.64,0.71,0.43,0.58,0.69
2,BERT,0.87,0.82,0.52,0.81,0.74,0.78,0.60,0.59,0.70
3,albert,0.53,0.61,0.54,0.78,0.54,0.79,0.50,0.47,0.62


In [7]:
idx = df[['imdb','covid','email','review','twitter','news', 'reddit', 'finance', 'paper']].idxmax(axis='rows')
best_alg = pd.DataFrame(list(zip(idx.index, df['alg'][idx])),
               columns =['dataset', 'alg'])
best_alg

,dataset,alg
0,imdb,BERT
1,covid,BERT
2,email,LSTM
3,review,BERT
4,twitter,BERT
5,news,albert
6,reddit,LSTM
7,finance,LSTM
8,paper,LSTM


In [72]:
wd = '/content/drive/MyDrive/UNI/COMPSCI 760/assignments/group project/scripts/output/'
knn = KNeighborsClassifier(n_neighbors=1)

def best_knn(f):

  mlr = pd.read_csv(wd+f)
  mlr_best_all = mlr.join(best_alg.set_index('dataset'), on='dataset', how='left')
  tr = mlr_best_all.iloc[:, :mlr_best_all.shape[1]].iloc[:6]
  ts = mlr_best_all.iloc[:, 1:mlr_best_all.shape[1]-1].iloc[6:]

  knn.fit(tr.iloc[:, 1:tr.shape[1]-1].values, tr['alg'])
  r = knn.predict(ts.values)

  return pd.DataFrame(list(zip(mlr_best_all['dataset'][6:].values, tr['alg'], r)), columns=['dataset', 'best_alg', 'pred_best_alg'])


In [73]:
# general meta
best_knn('mlr_general.csv')

,dataset,best_alg,pred_best_alg
0,reddit,albert,albert
1,finance,BERT,albert
2,paper,LSTM,LSTM


In [74]:
# hardness meta
best_knn('mlr_hardness.csv')

,dataset,best_alg,pred_best_alg
0,reddit,albert,LSTM
1,finance,BERT,albert
2,paper,LSTM,BERT


In [75]:
# readability meta
best_knn('mlr_rdblty.csv')

,dataset,best_alg,pred_best_alg
0,reddit,albert,albert
1,finance,BERT,albert
2,paper,LSTM,BERT


In [76]:
# word distribution meta
best_knn('mlr_word_dist.csv')

,dataset,best_alg,pred_best_alg
0,reddit,albert,LSTM
1,finance,BERT,albert
2,paper,LSTM,LSTM


In [77]:
# lexicon meta
best_knn('mlr_lex.csv')

,dataset,best_alg,pred_best_alg
0,reddit,albert,LSTM
1,finance,BERT,albert
2,paper,LSTM,BERT


In [78]:
# all meta
best_knn('meta_learning.csv')

,dataset,best_alg,pred_best_alg
0,reddit,albert,BERT
1,finance,BERT,albert
2,paper,LSTM,LSTM
